<a href="https://colab.research.google.com/github/Mohamed28112003/Muffakir/blob/main/DATA_GENERATION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  transformers
!pip install -U bitsandbytes
!pip install langchain_groq
!pip install langchain
!pip install -U langchain-community

In [2]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
import pandas as pd
from langchain.document_loaders import TextLoader
from langchain_groq import ChatGroq
import time
from langchain.schema import HumanMessage
from google.colab import files


Your final data frame

In [5]:
final_df = pd.DataFrame()


In [7]:

text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # Adjust chunk size based on content length
    chunk_overlap=0,  # No overlap needed
    separators=["Page number"] # Split at answer or next question
)





# Load documents from the specified directory
data_loader = TextLoader('/content/drive/MyDrive/Data/حقوق /عين شمس /رابعه/TXT/اجراءات_المحاكمة_وطرق_الطعن_فى_الاحكام_المصري.txt')
data = data_loader.load_and_split(text_splitter=text_splitter_recursive) # Load documents
print(f"Documents : {len(data)}.")


Documents : 796.


In [ ]:
print(data[296].page_content)

In [14]:
selected_data = data[297:796]
## DO NOT FORGET TO EDIT THE NUMBERS HERE AND ALSO NEGLECT ANY USEFUL PAGES

In [15]:
#  you can optimize this prompt if you need but keep it with the smae format please :)

prompt = """

Based on the following text, generate one Question and its corresponding Answer.
For each question you formulate, provide a comprehensive answer with details immediately following the question, separated by a vertical bar "|" provide only 2-3 questions. The context to analyze is as follows: <Context> {context} </Context>.
For Example :
## Context:
البعيد أو الغائي لقانون الإجراءات الجنائية يمثل ذات الهدف بالنسبة لكافة العلوم الجنائية.
رابعاً - وظيفة قانون الإجراءات الجنائية:
يمكن القول بأن الوظيفة الأساسية لهذا القانون هي تنظيم العدالة الجنائية؛ وتحقيق هذه
الغاية يتطلب الموائمة بين اعتبارين. يتمثل الاعتبار الأول في وضع آلية يُمكن من خلالها
تفعيل نصوص قانون العقوبات حماية للمصالح الاجتماعية العامة المتعددة، ويتجلى الاعتبار
الثاني في ضمان حقوق الفرد وحريته في مواجهة السلطات الإجرائية الجنائية.

# Your Response Sholud Be Like This format only do not add any thing else
Question : وظيفة قانون الإجراءات الجنائية | يمكن القول بأن الوظيفة الأساسية لهذا القانون هي تنظيم العدالة الجنائية؛ وتحقيق هذه
الغاية يتطلب الموائمة بين اعتبارين. يتمثل الاعتبار الأول في وضع آلية يُمكن من خلالها
تفعيل نصوص قانون العقوبات حماية للمصالح الاجتماعية العامة المتعددة، ويتجلى الاعتبار
الثاني في ضمان حقوق الفرد وحريته في مواجهة السلطات الإجرائية الجنائية.



"""

In [16]:
list_of_api = [
    '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',
    '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',
    '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',
    "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",
    "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",
    "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"
]

# REPLACE IT WITH YOU APIs

In [17]:
# Function to initialize the ChatGroq model with the current API key
def initialize_llm(api_key):
    return ChatGroq(
        api_key=api_key,
        model="llama-3.1-70b-versatile",
        temperature=0.7,
        max_tokens=1024
    )

In [19]:
# Initialize the ChatGroq model with the first API key
api_key_index = 0

llm = initialize_llm(list_of_api[api_key_index])

In [ ]:







results = []
i = 0

for doc in selected_data:
    Final_prompt = prompt.format(context=doc.page_content)
    messages = [HumanMessage(content=Final_prompt)]

    while True:
        try:
            response = llm.invoke(messages)
            break

        except Exception as e:
            error_message = str(e)
            print(f"Encountered an error: {error_message}")

            # Check if it's a rate limit error or service unavailable error
            if 'rate limit' in error_message.lower() or '429' in error_message:
                print(f"################################## Rate limit reached for API key ################################## {list_of_api[api_key_index]}. Switching API key...")
                api_key_index = (api_key_index + 1) % len(list_of_api)
                llm = initialize_llm(list_of_api[api_key_index])
                time.sleep(2)  # Wait before retrying
            elif '503' in error_message or 'Service Unavailable' in error_message:
                print("Service is unavailable, retrying after a delay...")
                time.sleep(5)  # Longer wait time for service errors
            else:
                raise  # Raise other unexpected errors

    # Split the response into individual question-answer pairs
    qa_pairs = response.content.split("Question :")
    print("Counter:", i)
    i += 1

    for qa in qa_pairs[1:]:  # Skip the first empty part
        question_answer = qa.split(" | ")
        if len(question_answer) == 2:
            question = question_answer[0].strip()
            answer = question_answer[1].strip()
            print("Question:", question)
            print("Answer:", answer)

            results.append({
                "context": doc.page_content,
                "question": question,
                "answer": answer
            })


In [ ]:

# Convert results to a DataFrame
df = pd.DataFrame(results)


In [ ]:
df.tail()

In [ ]:
df.shape

(340, 3)

In [ ]:


print("Context:\n", df.loc[2, "context"])

print("Question:\n", df.loc[2, "question"])
print("Answer:\n", df.loc[2, "answer"])


In [ ]:

pd.set_option('display.max_colwidth', None)


In [ ]:
df.tail(10)


In [ ]:
# CONCAT THE DF WITH TH FINAL_DF

final_df = pd.concat([final_df, df], ignore_index=True)



# Run this cell after you finish all the pages of the book

In [ ]:
# replace the book name


final_df.to_csv('Book Name.csv', index=False)

files.download('Book Name.csv')